## **LSTM and Random Forest Model**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

seed=3
torch.manual_seed(seed)
np.random.seed(seed)

### **Dataloader Part**
Load Dataset from google drive

In [4]:
import pandas as pd
training_url = 'https://drive.google.com/file/d/1uVq3dwf4CKwLd5U0Y8uMOjxkjjXmXfvO/view?usp=sharing'
testing_url='https://drive.google.com/file/d/1tELkS1DEUddnpKkguctOj6t8LvgJYDIz/view?usp=sharing'
training_file_id=training_url.split('/')[-2]
testing_file_id = testing_url.split('/')[-2]
teest_dwn_url='https://drive.google.com/uc?id=' + testing_file_id
train_dwn_url = 'https://drive.google.com/uc?id=' + training_file_id
train_data = pd.read_csv(train_dwn_url)
test_data = pd.read_csv(train_dwn_url)
train_head = train_data.head()
test_head = test_data.head()

train_head, test_head

(   group_id          arrive_time          depart_time  Fleet_Number   
 0    2375.0  2023-05-15 07:35:09  2023-05-15 07:35:49           934  \
 1    1505.0  2023-05-15 07:37:54  2023-05-15 07:39:15           934   
 2    2375.0  2023-05-15 07:53:50  2023-05-15 07:55:10           934   
 3    1505.0  2023-05-15 07:57:12  2023-05-15 07:58:15           934   
 4    2375.0  2023-05-15 08:24:28  2023-05-15 08:24:48           934   
 
    Timepoint  Time Difference Type Time      Group_Name dayofweek   
 0          1  0 days 00:03:26    depart  bayhi_to_oakes         M  \
 1          1  0 days 00:03:26    depart  bayhi_to_oakes         M   
 2          1  0 days 00:03:05    depart  bayhi_to_oakes         M   
 3          1  0 days 00:03:05    depart  bayhi_to_oakes         M   
 4          1  0 days 00:02:40    depart  bayhi_to_oakes         M   
 
               Sequence  
 0  05:00:00 - 07:45:00  
 1  05:00:00 - 07:45:00  
 2  07:50:00 - 07:55:00  
 3  07:55:00 - 08:00:00  
 4  08:05:00 -

### **Use Random Forest Regressor**
To predict the arrival time and delay of buses, we will approach the problem in two parts:

Arrival Time Prediction: We'll create a model to predict the arrival time of the buses at various stops or locations.
Delay Prediction: We'll calculate the delay as the difference between the predicted arrival time and the scheduled arrival time (if available in the data) or the historical average arrival time for that timepoint.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

def prepare_data(df):
    df['arrive_hour'] = pd.to_datetime(df['arrive_time']).dt.hour
    df['arrive_minute'] = pd.to_datetime(df['arrive_time']).dt.minute
    df['depart_hour'] = pd.to_datetime(df['depart_time']).dt.hour
    df['depart_minute'] = pd.to_datetime(df['depart_time']).dt.minute
    label_encoders = {}
    categorical_columns = ['group_id', 'Fleet_Number', 'dayofweek', 'Group_Name', 'Sequence']
    for col in categorical_columns:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])

    return df, label_encoders


train_data_prepared, label_encoders_train = prepare_data(train_data)
test_data_prepared, label_encoders_test = prepare_data(test_data)

features = ['group_id', 'Fleet_Number', 'dayofweek', 'Group_Name', 'Sequence', 'depart_hour', 'depart_minute']
target = ['arrive_hour', 'arrive_minute']

X_train, X_val, y_train, y_val = train_test_split(train_data_prepared[features], train_data_prepared[target], test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape


((17977, 7), (4495, 7), (17977, 2), (4495, 2))

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_val)

rmse_arrive_hour = math.sqrt(mean_squared_error(y_val['arrive_hour'], y_pred[:, 0]))
rmse_arrive_minute = math.sqrt(mean_squared_error(y_val['arrive_minute'], y_pred[:, 1]))

rmse_arrive_hour, rmse_arrive_minute


(0.1756247038181571, 5.331250589712805)

### **Use LSTM Model**

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

data = train_data
# Convert timestamps to numerical values or extract relevant features (like hour, minute)
data['hour'] = pd.to_datetime(data['arrive_time']).dt.hour
data['minute'] = pd.to_datetime(data['arrive_time']).dt.minute

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['group_id', 'Fleet_Number', 'dayofweek', 'Group_Name', 'Sequence', 'depart_hour', 'depart_minute']])
# seasonal pattern
# Monthly
# Convert the data to sequences
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences)

sequence_length = 5  # Number of previous time steps to consider
sequences = create_sequences(scaled_data, sequence_length)

# Split the data into features and target
X = sequences[:, :-1]  # All columns except the last one
y = sequences[:, -1, -1]  # Last column of the last sequence

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
import tensorflow as tf


print("TensorFlow version:", tf.__version__)

if tf.test.gpu_device_name():
    print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
    print("Failed to find the default GPU.")


TensorFlow version: 2.14.0
Failed to find the default GPU.


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Model architecture
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, verbose=1)
#use the random

Epoch 1/500
506/506 [==============================] - 8s 7ms/step - loss: 0.0935 - val_loss: 0.0890
Epoch 2/500
506/506 [==============================] - 3s 6ms/step - loss: 0.0878 - val_loss: 0.0815
Epoch 3/500
506/506 [==============================] - 3s 6ms/step - loss: 0.0809 - val_loss: 0.0781
Epoch 4/500
506/506 [==============================] - 3s 5ms/step - loss: 0.0795 - val_loss: 0.0761
Epoch 5/500
506/506 [==============================] - 3s 6ms/step - loss: 0.0753 - val_loss: 0.0715
Epoch 6/500
506/506 [==============================] - 3s 6ms/step - loss: 0.0712 - val_loss: 0.0672
Epoch 7/500
506/506 [==============================] - 4s 7ms/step - loss: 0.0678 - val_loss: 0.0644
Epoch 8/500
506/506 [==============================] - 4s 7ms/step - loss: 0.0652 - val_loss: 0.0619
Epoch 9/500
506/506 [==============================] - 4s 8ms/step - loss: 0.0627 - val_loss: 0.0620
Epoch 10/500
506/506 [==============================] - 4s 8ms/step - loss: 0.0615 - val_lo

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
train_loss = model.evaluate(X_train, y_train, verbose=0)
test_loss = model.evaluate(X_test, y_test, verbose=0)

print(f'Training Loss: {train_loss}')
print(f'Test Loss: {test_loss}')
